# Introduction to p8 python client for agentic systems
- Percolate is a database that allows agents to be built entirely in the data tier
- However either for testing or to orchastrate more complex agents in code, we provide a python guide
- This giude assumes that you have followed the guides to add some initial data and tools
- to learn more see here

In [ ]:
import percolate as p8

# Agents are simple in Percolate
- They are just Pydantic objects with references to extenral tools in your function repository
- You can provide structure and you can use this structure to save and search data in the database for your agent


In [ ]:
from pydantic import BaseModel,Field
import typing
from percolate.models import DefaultEmbeddingField

class MyFirstAgent(BaseModel):
    """You are an agent that provides the information you are asked and a second random fact"""
    #because it has no config it will save to the public database schema
    
    name: str = Field(description="Task name")
    #the default embedding field just settgs json_schema_extra.embedding_provider so you can do that yourself
    description:str = DefaultEmbeddingField(description="Task description")
    
    @classmethod
    def get_functions(cls):
        """i return a list of functions by key stored in the database"""
        return {}

In [ ]:
p8.Agent(MyFirstAgent).run("what is the capital of ireland")

In [ ]:
# 0.test registering api fuctions and later apis
# 1.add the functions that we have in get_funtions on register external functions in this context are added as inline functions in this approach
# 2.test other models including grok xai
# 3.external functions are searchable by plans
# 4.test inline functions even though we dont use them
# 5.test restoring a function from the database as above
# 6.test planning over entities / functions
# 7.test using native functions
# 8.add the database interface for function calls - lots of tests for different api responses
# 9.test loading entities and their functions
#10.test crud

## Behind the scenes
- we created a simple rest api wrapper instead of adding various api python clients
- you can can iterate over these check the response of each model for which you have an API key defined
- you should first check the langauge model is in your data store - either you have added the API token there or in the env var given by the reference to the env
- we will then iterate and ask the same question first with tools and then without tools to see what the payloads look like for testing
- **This is illustrated in `percolate.services.llm._check_all`**
- the high level wrapper we use parsers to a single response scheme

In [9]:
from percolate.services.llm import _check_all, LanguageModel
#some example response are in the tests

### asking without tools

In [2]:
gem = LanguageModel('gemini-1.5-flash')
gem.ask("What is the capital of ireland", system_prompt="Give me a random fact when answering my question")

2025-01-27 07:34:03.439 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:150 - invoking model gemini-1.5-flash


AIResponse(id='fe3554ae-dcaa-11ef-889b-7606330c2360', model_name='gemini-1.5-flash', tokens=None, tokens_in=16, tokens_out=30, tokens_other=None, session_id=None, role='model', content='The capital of Ireland is Dublin.\n\nDid you know that Ireland is the only island in the world with two official languages, Irish and English?\n', status='RESPONSE', tool_calls=[], tool_eval_data=None)

In [2]:
claude = LanguageModel('claude-3-5-sonnet-20241022')
claude.ask("What is the capital of ireland", system_prompt="Give me a random fact when answering my question")

2025-01-27 07:37:45.400 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:150 - invoking model claude-3-5-sonnet-20241022


AIResponse(id='843d1b18-dcab-11ef-b21f-7606330c2360', model_name='claude-3-5-sonnet-20241022', tokens=None, tokens_in=23, tokens_out=106, tokens_other=None, session_id=None, role='assistant', content="The capital of Ireland is Dublin. It's both the largest city and the capital, located on the eastern coast of the country at the mouth of the River Liffey.\n\nRandom fact: While we're talking about Ireland, did you know that the submarine was invented by an Irishman? John Philip Holland, born in County Clare, Ireland, created the first submarine to be formally commissioned by the U.S. Navy in 1900. His design became the foundation for modern submarine development!", status='RESPONSE', tool_calls=[], tool_eval_data=None)

In [8]:
xai = LanguageModel('grok-2-latest')
xai.ask("What is the capital of ireland", system_prompt="Give me a random fact when answering my question")

2025-01-27 07:42:09.993 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:150 - invoking model grok-2-latest


AIResponse(id='208abf2a-dcac-11ef-b21f-7606330c2360', model_name='grok-2-1212', tokens=None, tokens_in=26, tokens_out=46, tokens_other=None, session_id=None, role='assistant', content='The capital of Ireland is Dublin. \n\nRandom fact: Did you know that Dublin is home to the famous Guinness Brewery, which was founded in 1759 and is one of the most visited tourist attractions in Ireland?', status='RESPONSE', tool_calls=[], tool_eval_data=None)

In [9]:
deepseek = LanguageModel('deepseek-chat')
deepseek.ask("What is the capital of ireland", system_prompt="Give me a random fact when answering my question")

2025-01-27 07:42:40.933 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:150 - invoking model deepseek-chat


AIResponse(id='33c8598a-dcac-11ef-b21f-7606330c2360', model_name='deepseek-chat', tokens=None, tokens_in=19, tokens_out=55, tokens_other=None, session_id=None, role='assistant', content='The capital of Ireland is Dublin. \n\nRandom fact: Dublin is home to the famous Guinness Storehouse, where you can learn about the history of Guinness beer and enjoy a pint with a panoramic view of the city from the Gravity Bar. Cheers! 🍻', status='RESPONSE', tool_calls=[], tool_eval_data=None)

### asking with tools
- we create a fictitious tool first - this tested the response in a simple way

In [1]:
from percolate.services.llm import _check_all, LanguageModel

fns =[{
  "name": "get_weather",
  "description": "Get the weather forecast for a specific city and date",
  "parameters": {
    "type": "object",
    "properties": {
      "city": {
        "type": "string",
        "description": "The city for which to get the weather forecast"
      },
      "date": {
        "type": "string",
        "description": "The date for the weather forecast (YYYY-MM-DD)"
      }
    },
    "required": ["city", "date"]
  }
}]



In [2]:
claude = LanguageModel('claude-3-5-sonnet-20241022')
claude.ask("What is the weather in dublin tomorrow", functions=None)


2025-01-27 08:39:35.873 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:169 - invoking model claude-3-5-sonnet-20241022
2025-01-27 08:39:35.874 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:240 - request data={'model': 'claude-3-5-sonnet-20241022', 'temperature': 0.01, 'messages': [{'role': 'user', 'content': 'What is the weather in dublin tomorrow'}], 'max_tokens': 8192}


AIResponse(id='2731af0c-dcb4-11ef-b42b-7606330c2360', model_name='claude-3-5-sonnet-20241022', tokens=None, tokens_in=14, tokens_out=87, tokens_other=None, session_id=None, role='assistant', content='I am not able to provide real-time weather forecasts or predictions. To get accurate weather information for Dublin tomorrow, I recommend:\n\n1. Checking a weather website or app (like Met Éireann for Ireland)\n2. Using your local weather service\n3. Checking news websites with weather sections\n\nThese sources will give you up-to-date and accurate weather forecasts for Dublin.', status='RESPONSE', tool_calls=[], tool_eval_data=None)

In [4]:
gem = LanguageModel('gemini-1.5-flash')
gem.ask("What is the weather in dublin", functions=fns)


2025-01-27 08:39:51.748 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:169 - invoking model gemini-1.5-flash
2025-01-27 08:39:51.750 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:240 - request data={'contents': [{'role': 'user', 'parts': {'text': 'What is the weather in dublin'}}], 'tool_config': {'function_calling_config': {'mode': 'ANY'}}, 'tools': [{'function_declarations': [{'name': 'get_weather', 'description': 'Get the weather forecast for a specific city and date', 'parameters': {'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'The city for which to get the weather forecast'}, 'date': {'type': 'string', 'description': 'The date for the weather forecast (YYYY-MM-DD)'}}, 'required': ['city', 'date']}}]}]}


AIResponse(id='2faba750-dcb4-11ef-b42b-7606330c2360', model_name='gemini-1.5-flash', tokens=None, tokens_in=81, tokens_out=17, tokens_other=None, session_id=None, role='model', content='', status='TOOL_CALLS', tool_calls=[{'function': {'name': 'get_weather', 'arguments': {'city': 'dublin', 'date': '2024-02-29'}}}], tool_eval_data=None)

In [3]:
gmin = LanguageModel('gpt-4o-mini')
gmin.ask("What is the weather in dublin tomorrow", functions=fns)


2025-01-27 08:29:41.358 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:168 - invoking model gpt-4o-mini
2025-01-27 08:29:41.359 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:239 - request data={'model': 'gpt-4o-mini', 'messages': [{'role': 'user', 'content': 'What is the weather in dublin tomorrow'}], 'tools': [{'type': 'function', 'function': {'name': 'get_weather', 'description': 'Get the weather forecast for a specific city and date', 'parameters': {'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'The city for which to get the weather forecast'}, 'date': {'type': 'string', 'description': 'The date for the weather forecast (YYYY-MM-DD)'}}, 'required': ['city', 'date']}}}], 'temperature': 0.01}


AIResponse(id='c407fc98-dcb2-11ef-94f1-7606330c2360', model_name='gpt-4o-mini-2024-07-18', tokens=None, tokens_in=82, tokens_out=25, tokens_other=None, session_id=None, role='assistant', content='', status='TOOL_CALLS', tool_calls=[{'id': 'call_hZgUO0LgPAklKbtlnl4nGc0V', 'type': 'function', 'function': {'name': 'get_weather', 'arguments': '{"city":"Dublin","date":"2023-10-06"}'}}], tool_eval_data=None)

In [5]:
import percolate as p8
from pydantic import BaseModel,Field
import typing
from percolate.models import DefaultEmbeddingField

class MyFirstAgent(BaseModel):
    """You are an agent that provides the information you are asked and a second random fact"""
    #because it has no config it will save to the public database schema
    
    name: str = Field(description="Task name")
    #the default embedding field just settgs json_schema_extra.embedding_provider so you can do that yourself
    description:str = DefaultEmbeddingField(description="Task description")
    
    @classmethod
    def get_model_functions(cls):
        """i return a list of functions by key stored in the database"""
        return {
            'get_pet_findByStatus': "a function i used to look up petes based on their status",
            'p8_About' : 'a "native" database function that gives me general information about percolate'
        }
 

In [ ]:
# we register APIs as follows

In [ ]:
from percolate.utils.ingestion import add 
add.add_api('swagger_test', 'https://petstore.swagger.io/v2/swagger.json', verbs='get')

In [ ]:
# we can check the function is there

In [ ]:
from percolate.models.p8 import Function

p8.repository(Function).get_by_name(['get_pet_findByStatus'], as_model=True)

In [ ]:
#using
agent = p8.Agent(MyFirstAgent)
agent.run("can you find available pets",limit=2)

In [ ]:
#using native functions -> p8_About
p8.Agent(MyFirstAgent).run("what can you tell me about percolate")